# NPT模拟

In [1]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import MDAnalysis as mda
import nglview as nv
import numpy as np

In [2]:
# 与处理蛋白
fixer=PDBFixer(pdbid='5xg0')
fixer.removeChains(chainIds=["B","C"])
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(keepWater=False)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(padding=10*angstroms,positiveIon='Na+',negativeIon='Cl-')
PDBFile.writeFile(fixer.topology,fixer.positions,open('5xg0_process.pdb','w'))

In [3]:
# 查看生成的pdb文件
view=nv.show_mdanalysis(mda.Universe('5xg0_process.pdb'))
view.add_licorice('water',opacity=0.35)
view.add_spacefill('ion')
view.center('all')
view

NGLWidget()

In [4]:
# 模拟设置 
pdbfile='5xg0_process.pdb' # 处理好的pdb文件

nonbondedmethod=PME   # 处理非相互作用力 NoCutoff | CutoffNonPeriodic | CutoffPeriodic | Ewald | PME | LJPME 
nonbondedcutoff=1*nanometer  # 非键相互作用的阶段距离
constraints= HBonds   # 施加约束 None, HBonds, AllBonds, or HAngles
removecmmotion=True    # 是否移除质心运动
rigidwater = True      # 是否使用刚性水
ewaldErrorTolerance = 0.0005  
constraintTolerance = 0.000001     
hydrogenMass = 1.5*amu   # 氢质量

dt = 0.002*picoseconds   # 步长
temperature = 300*kelvin   # 模拟温度
friction = 1.0/picosecond  # 温度耦合时间
pressure = 1.0*atmospheres  # 压力
barostatInterval = 25       # 压力耦合步数

annealtemp=np.array([300,280,300,320,340,360,340,320,300])   # 模拟退火温度设置
annealstep=10000     # 每个温度步数
equilibrationSteps=10000    # 平衡步长
steps=20000                # 模拟步长
 
dcd=True                    # 是否保存dcd文件
dcdstep=1000                # 多少步保存一次轨迹
chk=True                    # 是否保存检查点文件
chkstep=1000                # 多少步保存一次检查点文件
data=stdout               # 以文本格式还是直接输出文件 stdout | 'log.txt
datastep=100               # 多少步输出一次运行信息

dcdReporter = DCDReporter('trajectory.dcd', dcdstep)
dataReporter = StateDataReporter(data,datastep, totalSteps=steps,
    step=True, speed=True, progress=True, potentialEnergy=True, temperature=True,density=True,volume=True,separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', chkstep)

platform=Platform.getPlatformByName("CUDA")  # 模拟平台 CUDA | CPU |OPENGL
properties={"DeviceIndex":"0","Precision":"single"}  # 选择cuda时所使用的显卡，以及需要的精度类型 single double

In [5]:
pdb=PDBFile(pdbfile)
forcefield=ForceField('amber14-all.xml','amber14/tip3p.xml')
system=forcefield.createSystem(pdb.topology,nonbondedMethod=nonbondedmethod,
                                nonbondedCutoff=nonbondedcutoff,
                                constraints=constraints,
                                removeCMMotion=removecmmotion,
                                rigidWater=rigidwater,
                                ewaldErrorTolerance=ewaldErrorTolerance, 
                                hydrogenMass=hydrogenMass
                                )
# 另一种控温
# system.addForce(AndersenThermostat(temperature,friction))
# integrator = VerletIntegrator(0.002*picoseconds)
system.addForce(MonteCarloBarostat(pressure,temperature,barostatInterval))

5

In [6]:
integrator=LangevinIntegrator(temperature,friction,dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation=Simulation(pdb.topology,system,integrator,platform)
simulation.context.setPositions(pdb.positions)

In [7]:
# 最小化
simulation.minimizeEnergy()

In [8]:
# 平衡
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

In [9]:
simulation.currentStep = 0
for i in annealtemp:
    integrator.setTemperature(i)
    simulation.context.setParameter(MonteCarloBarostat.Temperature(),temperature)
    # simulation.context.setParameter(MonteCarloBarostat.Pressure(), pressure)
    simulation.step(annealstep)


#"Temperature (K)"	"Box Volume (nm^3)"	"Density (g/mL)"
299.18034603612347	436.87100915048524	1.014442656029824
301.075326304945	436.3301306825172	1.0157001676043207
298.9498951418811	436.5513872310857	1.0151853821288008
298.1007472052585	437.81320125398213	1.0122595335081095
298.2662388834782	436.9297402633015	1.0143062969299814
298.4262656922621	436.9297402633015	1.0143062969299814
300.9637287004054	436.80096248627314	1.0146053349847535
299.94911451142934	437.118037969631	1.0138693633499472
299.40174932838795	437.1006120264351	1.0139097834030146
299.91388369766344	436.92319296706654	1.014321496314921
299.103942279633	437.13267352875073	1.013835418175163
299.8685601146877	437.13267352875073	1.013835418175163
297.37842746908444	436.02440887951434	1.0164123334377617
300.23403092425366	437.34258883786794	1.0133487983475222
301.1020713905834	437.5936060186661	1.0127675102413254
301.14090252461375	437.5936060186661	1.0127675102413254
300.9609329375407	438.016124131782	1.0117905767590238
29

In [33]:
# 模拟
if dcd:
    simulation.reporters.append(dcdReporter)
else:
    pass
simulation.reporters.append(dataReporter)
if chk:
    simulation.reporters.append(checkpointReporter)
else:
    pass
simulation.currentStep = 0
simulation.step(steps)
positions=simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology,positions,open("lastframe.pdb","w"))

#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Box Volume (nm^3)"	"Density (g/mL)"	"Speed (ns/day)"
0.5%	100	-560294.605358453	300.941876441936	440.0772236257678	1.0141894503545472	0
1.0%	200	-560252.105358453	298.6282782323613	440.0772236257678	1.0141894503545472	239
1.5%	300	-560908.2044523465	299.59427773279504	439.9112745120185	1.0145720362308623	245
2.0%	400	-560928.9804884265	299.8325381211868	438.62819090281516	1.0175398818391541	245
2.5%	500	-561091.7288735718	300.0802252146156	439.4552709810704	1.015624813297092	246
3.0%	600	-561026.9942469536	299.32038496225096	439.07673799952556	1.0165003948422673	246
3.5%	700	-561382.5170352892	302.6527606554545	439.63787090802606	1.0152029819922967	245
4.0%	800	-559764.0170352892	299.12904449473086	439.63787090802606	1.0152029819922967	245
4.5%	900	-560294.7051030449	303.04644099312367	439.12021745212616	1.0163997461383831	244
5.0%	1000	-560317.3193678362	302.9022913341364	440.76342684028924	1.0126105079591763	205
5

In [34]:
# 查看生成的pdb文件
view=nv.show_mdanalysis(mda.Universe('lastframe.pdb'))
view.add_licorice('water',opacity=0.35)
view.add_spacefill('ion')
view.center('all')
view

NGLWidget()

In [35]:
traj=nv.show_mdanalysis(mda.Universe('lastframe.pdb','trajectory.dcd'))
# traj.add_licorice('water',opacity=0.35)
# traj.add_spacefill('ion')
traj

/home/casea/CASEADATA/software/Anaconda/envs/openmm/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


NGLWidget(max_frame=19)